In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [8]:
!git clone 'https://github.com/Beomi/KoAlpaca.git'

Cloning into 'KoAlpaca'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 248 (delta 84), reused 101 (delta 69), pack-reused 126
Receiving objects: 100% (248/248), 31.60 MiB | 15.77 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [ ]:
!pip install -r /content/KoAlpaca/requirements.txt

In [2]:
!pip install -U torch transformers tokenizers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00


In [3]:
import torch

torch.cuda.is_available()

True

## 테스트 데이터

In [4]:
!wget -q https://github.com/Beomi/KoAlpaca/raw/main/ko_alpaca_data.json

import json

with open('ko_alpaca_data.json', 'r') as f:
    dataset = json.load(f)

In [2]:
dataset[:10]

[{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
  'input': '',
  'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'},
 {'instruction': '세 가지 기본 색은 무엇인가요?',
  'input': '',
  'output': '기본 색은 빨강, 파랑, 노랑입니다.'},
 {'instruction': '원자의 구조를 설명하세요.',
  'input': '',
  'output': '원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있고 전자는 주변에 있습니다.'},
 {'instruction': '대기 오염을 어떻게 줄일 수 있나요?',
  'input': '',
  'output': '대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.'},
 {'instruction': '어려운 결정을 내려야 했던 때를 설명하세요.',
  'input': '',
  'output': '제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다. 다양한 선택지가 있었지만 나의 관심사와 비전에 가장 부합하는 경력을 뽑는데 어려움이 있었습니다. 이 때 여러 사람들의 조언과 현재 상황에 대한 깊은 고민 끝에 적절한 선택을 내리게 되었습니다.'},
 {'instruction': '홀수 중 하나를 밝히세요.',
  'input': '트위터, 인스타그램, 텔레그램',
  'output': '텔레그램입니다.'},
 {'instruction': '다음 분수가 1/4에 해당하는 이유를 설명하세요.',
  'input': '4/16',
  'output': '분모와 분자를 모두 4로 나눌 수 있기 때문에 1/4에 해당합니다.'},
 {'instruction': '중

In [5]:
instruction_list = [data['instruction'] for data in dataset[:50]]
input_list = [data['input'] for data in dataset[:50]]
output_list = [data['output'] for data in dataset[:50]]

In [6]:
import pandas as pd

test_df = pd.DataFrame({
    'instruction': instruction_list,
    'input': input_list,
    'output': output_list
})

test_df.head()

,instruction,input,output
0,건강을 유지하기 위한 세 가지 팁을 알려주세요.,,"세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는..."
1,세 가지 기본 색은 무엇인가요?,,"기본 색은 빨강, 파랑, 노랑입니다."
2,원자의 구조를 설명하세요.,,"원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있..."
3,대기 오염을 어떻게 줄일 수 있나요?,,"대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기..."
4,어려운 결정을 내려야 했던 때를 설명하세요.,,제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다....


## KoAlpaca

KoAlpaca-13B-LORA

In [5]:
!pip install -q bitsandbytes datasets loralib sentencepiece git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git gradio==3.20.0 tenacity

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.

In [6]:
from peft import PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM

base = 'decapoda-research/llama-13b-hf'
finetuned_13B = 'beomi/KoAlpaca-13B-LoRA'

tokenizer = LlamaTokenizer.from_pretrained(base)
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

model_LoRA13B = LlamaForCausalLM.from_pretrained(
    base,
    load_in_8bit=True,
    device_map="auto",
)

model_LoRA13B = PeftModel.from_pretrained(model_LoRA13B, finetuned_13B, device_map={'': 0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [7]:
model_LoRA13B

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 5120, padding_idx=31999)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(in_features=5120, out_features=5120, bias=False)
        

In [7]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
    ),
}

In [8]:
def gen(model, prompt, user_input=None, max_new_tokens=128, temperature=0.5):
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
    # if user_input:
    #     x = f"""Below is a history of instructions that describe tasks, paired with an input that provides further context. Write a response that appropriately completes the request by remembering the conversation history.
    #     아래는 작업을 설명하는 지시 사항의 히스토리와 추가 컨텍스트를 제공하는 입력이 짝을 이루고 있습니다. 대화 기록을 기억하여 요청을 적절히 완료하는 응답을 작성하세요.

    #     ### Input: {input}

    #     ### Instruction: {prompt}

    #     ### Response:"""
    # else:
    #     x = f"""Below is a history of instructions that describe tasks, paired with an input that provides further context. Write a response that appropriately completes the request by remembering the conversation history.
    #     아래는 작업을 설명하는 지시 사항의 히스토리와 추가 컨텍스트를 제공하는 입력이 짝을 이루고 있습니다. 대화 기록을 기억하여 요청을 적절히 완료하는 응답을 작성하세요.

    #     ### Instruction: {prompt}

    #     ### Response:"""

    input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
    gen_tokens = model.generate(
      input_ids=input_ids, 
      max_new_tokens=max_new_tokens, 
      num_return_sequences=1, 
      temperature=temperature,
      no_repeat_ngram_size=6,
      do_sample=True,
    )

    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    
    return gen_text.replace(x, '')

In [22]:
# Example usage:
# prompt = "입력받은 수 이하의 소수(Prime number)를 순서대로 출력하는 파이썬 코드를 작성해주세요."
prompt = "안녕하세요"
generated_text = gen(model_LoRA13B, prompt)
print(generated_text)

 안념입니다.

    # 추가적인 질문

    # 답변 추출

    # 인식 없는 문장 처리

    # 텍스트 추����

    # 문장 형�� 추����

    # NLP 모델 ���� 구 추


In [13]:
def isPrime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

print(is_prime(1))
print(is_prime(4))
print(is_prime(5))
print(is_prime(11))

False
False
True
True


In [ ]:
KoAlpaca_LoRA13B_output = []
for i in test_df.index[:30]:
  print(f'{i + 1}th instruction: {test_df.iloc[i]["instruction"]}')
  if not test_df.iloc[i]['input']:
    output = gen(model_LoRA13B, prompt)
  else:
    output = gen(model_LoRA13B, prompt, test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}')
  KoAlpaca_LoRA13B_output.append(output)

test_df['KoAlpaca_LoRA13B_output'] = KoAlpaca_LoRA13B_output

In [ ]:
test_df.head()

KoAlpaca-30B-LORA

In [ ]:
base = 'decapoda-research/llama-13b-hf'
finetuned_30B = 'beomi/KoAlpaca-30B-LoRA'

tokenizer = LlamaTokenizer.from_pretrained(base)
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

model_LoRA30B = LlamaForCausalLM.from_pretrained(
    base,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    # device_map="auto",
)

model_LoRA30B = PeftModel.from_pretrained(model_LoRA30B, finetuned_30B, device_map={'': 0})

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [ ]:
KoAlpaca_LoRA30B_output = []
for i in test_df.index:
  print(f'{i + 1}th iteration}')
  if not test_df.iloc[i]['input']:
    output = gen(model_LoRA30B, prompt)
  else:
    output = gen(model_LoRA30B, prompt, test_df.iloc[i]['input'])
  print(output)
  KoAlpaca_LoRA30B_output.append(output)

test_df['KoAlpaca_LoRA30B_output'] = KoAlpaca_LoRA30B_output

In [ ]:
test_df.head()

KoAlpaca-Polyglot-5.8B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B")
model_Polyglot58B = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B")
# model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B")

In [ ]:
KoAlpaca_Polyglot58B_output = []
for i in test_df.index:
  print(f'{i + 1}th iteration')
  if not test_df.iloc[i]['input']:
    output = gen(model_Polyglot58B, prompt)
  else:
    output = gen(model_Polyglot58B, prompt, test_df.iloc[i]['input'])
  print(output)
  KoAlpaca_Polyglot58B_output.append(output)

test_df['KoAlpaca_Polyglot5.8B_output'] = KoAlpaca_Polyglot58B_output

In [ ]:
KoAlpaca_LoRA13B_output = []
for i in test_df.index[:30]:
  print(f'{i + 1}th instruction: {test_df.iloc[i]["instruction"]}')
  if not test_df.iloc[i]['input']:
    output = gen(model_LoRA13B, prompt)
  else:
    output = gen(model_LoRA13B, prompt, test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}')
  KoAlpaca_LoRA13B_output.append(output)

test_df['KoAlpaca_LoRA13B_output'] = KoAlpaca_LoRA13B_output

In [2]:
# PROMPT_DICT = {
#     "prompt_input": (
#         "Below is an instruction that describes a task, paired with an input that provides further context.\n"
#         "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
#         "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
#         "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
#     ),
#     "prompt_no_input": (
#         "Below is an instruction that describes a task.\n"
#         "아래는 작업을 설명하는 명령어입니다.\n\n"
#         "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
#         "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
#     ),
# }

In [3]:
# def gen(prompt, user_input=None, max_new_tokens=128, temperature=0.5):
#     if user_input:
#         x = PROMPT_DICT['prompt_input'].format(instruction=prompt, input=user_input)
#     else:
#         x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
#     input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
#     gen_tokens = model.generate(
#         input_ids, 
#         max_new_tokens=max_new_tokens, 
#         num_return_sequences=1, 
#         temperature=temperature,
#         no_repeat_ngram_size=6,
#         do_sample=True,
#     )
#     gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    
#     return gen_text.replace(x, '')

In [10]:
# dataset_path = '/content/KoAlpaca/KoAlpaca_v1.1.jsonl'

# with open(dataset_path, 'r', encoding = 'utf-8') as f:
#   print(f)

<_io.TextIOWrapper name='/content/KoAlpaca/KoAlpaca_v1.1.jsonl' mode='r' encoding='utf-8'>


In [9]:
# # Example usage:
# prompt = "양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?"
# generated_text = gen(prompt)
# print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



양파은 어떤 식물 부위인가요?
 
 
고구마는 어떤 뿌리인가요?

답변에 따른 명령어를 작성하세요.
### 입력: 양파는 어떤 종류의 식물에서 생성되는 단백질, 지질 및 섬유질과 같은 부분인가요?

# 응답: 양파는 어떤 종류 이상의 식물에서 생성되는데, 이에는 Fruits, Vegetables, Seafood, Compositions, Lipid peroxidation 등이 포함됩니다. 따라서, 양파는 일반적으로 뿌리 또는 줄기 중 하나
